## 4.1 Introduction

### pp. 103
Eq. (4.28) looks wierd, as it seems that Gaussian plays no role in proof. No. This is because $\log p(x)$ takes a quadratic form (check Eq. (4.24)), and in this Theorem, we assume that $q$ and $p$ match in terms of second order moments.

## 4.2 Gaussian Discriminat Analysis

This section gives a very detailed overview of all generative classifiers using Gaussian assumption. Basically, five things are discussed. 1) pure QDA. 2) pure LDA (here LDA has nothing to do with Fisher, nor latent Dirichlet allocation). 3) ML estimate. 4) regularized LDA. 5) feature selection.

### pp. 109
There are some problems and confusions with the SVD trick for Regularized LDA.

1. When it says using prior of the form $\mathrm{IW}(\mathrm{diag}(\Sigma))$, we actually have to compute the MLE of $\Sigma$ first, and then use its diag values as paramters for the prior. This is very different from the typical ridge regression, where we use some fixed diagonal matrix as prior. Here, values along the diagonal are different.
2. When evaluating the the inverse of regularized $\Sigma$, it's possible to compute its inverse, since Eq. (4.54) is sum of a positive definite matrix and a positive semidefinite matrix. (I assume all diagonal values of MLE $\Sigma$ are positive), and here SVD trick is just to speed up. Check pp. 659 (18.3.5) of **The Elements of Statistical Learning (2nd edition)** (abbreviated as ESLII below).
3. bottom of pp. 109, the author says we can recover original mean. It's not because $V^TV=VV^T=I$. Instead, only $V^TV=I$. But here, we seem to need the later. Although $VV^T \ne I$, we are still fine because $\mu$ is mean of all rows of $X$, and by this SVD, you can see that every row is linear combinations of columns of $V$. Thus, $\mu$ falls into the subspace defined by $V$, and $V\mu_z = VV^T \mu = V (V^TV)^{-1} V^T \mu= \mu$ since $ V (V^TV)^{-1} V^T$ is projection (check Eq. 7 on pp. 210 of Strang's Introduction to Linear Algebra, 4th edition) and $\mu$ is already in that subspace
4. on pp. 110, Eq (4.60) and Eq. (4.61) needs some justification. It's not very obvious why taking diag and multiplication by $V$ and $V^T$ on two sides can commute. Actually, it's wrong. The correct way is to use Woodbury inversion lemma.
    * It should be obvious that (4.60) can't be equal to (4.54), since (4.60) is not full rank, yet (4.60) is.

In [1]:
import numpy as np
from scipy.linalg import svd, inv
rng_state = np.random.RandomState(seed=0)

In [2]:
n = 3
p = 10
X = rng_state.randn(n, p)
U, S, Vh = svd(X, full_matrices=False)
Z = U.dot(np.diag(S))
mean_X = np.mean(X, axis=0, keepdims=True).T
mean_Z = np.mean(Z, axis=0, keepdims=True).T
mean_Z_debug = Vh.dot(mean_X)
assert np.allclose(mean_Z, mean_Z_debug)

X_c = X - mean_X.T
Z_c = Z - mean_Z.T

cov_mle_X = 1/n*X_c.T.dot(X_c)
cov_mle_X_debug = np.cov(X, rowvar=False, ddof=0)
cov_mle_Z = 1/n*Z_c.T.dot(Z_c)
cov_mle_Z_decompose = 1/np.sqrt(n)*Z_c.T
cov_mle_Z_composed = cov_mle_Z_decompose.dot(cov_mle_Z_decompose.T)
cov_mle_Z_debug = np.cov(Z, rowvar=False, ddof=0)
cov_mle_X_by_Z = Vh.T.dot(cov_mle_Z).dot(Vh)
assert np.allclose(cov_mle_X, cov_mle_X_by_Z)
assert np.allclose(cov_mle_X, cov_mle_X_debug)
assert np.allclose(cov_mle_Z, cov_mle_Z_composed)
assert np.allclose(cov_mle_Z, cov_mle_Z_debug)


lam = 0.5  # regularization factor
cov_reg_X_correct = lam*np.diag(np.diag(cov_mle_X)) + (1-lam)*cov_mle_X
cov_reg_Z = lam*np.diag(np.diag(cov_mle_Z)) + (1-lam)*cov_mle_Z
cov_reg_X_trick = Vh.T.dot(cov_reg_Z).dot(Vh)
print('mean abs diff between cov_reg_X_correct and one obtained using trick', abs(cov_reg_X_correct - cov_reg_X_trick).mean())
cov_reg_X_correct_inv = inv(cov_reg_X_correct)

# you can't since Eq. (4.60) matrix is not full rank, yet (4.54) is.
# print('mean abs diff between inv of cov_reg_X_correct and one obtained using trick', abs(cov_reg_X_correct_inv - inv(cov_reg_X_trick)).mean())

# let's do it the correct way.
# using woodbury formula, where A is lam*np.diag(np.diag(cov_mle_X)), D^{-1} is (1-lam)*I, and B = V*cov_mle_Z_decompose,
# C = cov_mle_Z_decompose.T*Vh. It's important to use this D^{-1}, rather than cov_mle_Z, since cov_mle_Z is not invertible.

cov_reg_X_part1 = lam*np.diag(np.diag(cov_mle_X))
Ainv = inv(cov_reg_X_part1)
B = Vh.T.dot(cov_mle_Z_decompose)
C = cov_mle_Z_decompose.T.dot(Vh)
Dinv = (1-lam)*np.eye(n)

new_inv = inv(inv(Dinv) + C.dot(Ainv).dot(B))
assert new_inv.shape == (n,n)
cov_reg_inv_by_woodbury = Ainv - Ainv.dot(B).dot(new_inv).dot(C).dot(Ainv)
assert cov_reg_inv_by_woodbury.shape == cov_reg_X_correct_inv.shape
assert np.allclose(cov_reg_inv_by_woodbury, cov_reg_X_correct_inv, atol=1e-10)

beta_naive = cov_reg_X_correct_inv.dot(mean_X)
beta_trick = Vh.T.dot(inv(cov_reg_Z)).dot(mean_Z)
beta_woodbury = cov_reg_inv_by_woodbury.dot(mean_X)
assert np.allclose(beta_naive, beta_woodbury) and beta_naive.shape == beta_woodbury.shape == beta_trick.shape
print('difference bewteen the one given by Eq. 4.63 and the one using naive approach\n', beta_trick-beta_naive)
print('difference bewteen the one given by woodbury approach and the one using naive approach\n', beta_woodbury-beta_naive)

mean abs diff between cov_reg_X_correct and one obtained using trick 0.14946654667
difference bewteen the one given by Eq. 4.63 and the one using naive approach
 [[ -3.07933880e-01]
 [ -9.01774381e+00]
 [ -2.13349637e+02]
 [ -5.89202525e-01]
 [ -2.99615080e+00]
 [  1.03976516e+00]
 [ -5.26540419e+00]
 [  7.41557472e+02]
 [ -1.19513324e+00]
 [  1.19362849e-01]]
difference bewteen the one given by woodbury approach and the one using naive approach
 [[ -3.60822483e-15]
 [ -6.21724894e-14]
 [  3.12638804e-13]
 [  3.10862447e-15]
 [  3.19744231e-14]
 [ -3.84137167e-14]
 [ -3.73034936e-14]
 [  5.68434189e-13]
 [  1.22124533e-14]
 [  2.39808173e-14]]


In pp. 660 of ESLII, the author says that SVD trick can be applied to any linear model with quadratic penalties. But the details are omitted. I guess this is more or less like kernel trick, where we all know the general idea, but derivation for each particular case needs some work, especially when the paramterization of the model is not good (here for regularized LDA, we have redundant parameters, such as symmetric terms for covariance matrix or precision matrix). This page also gives some basic reason why this works.

> Geometrically, we are rotating the features to a coordinate system in which all but the first $N$ coordinates are zero. Such rotations are allowed since the quadratic penalty is invariant under rotations, and linear models are equivariant.

* "quadratic penalty is invariant under rotations" basically means that the square of norm (thus quadratic) of some penalty is not changed under rotation, or more generally, orthogonal transformation.
* "linear models are equivariant" sounds reasonble, although I don't understand what equivariant means exactly.

Essentially, all these tricks are orthogonal transformation, exploiting invariance properties of quadratic regularization. Check [Efficient quadratic regularization for expression arrays](http://dx.doi.org/10.1093/biostatistics/kxh010) for detail. Looks that the theorem is easy, and more understanable than the presentation in MLAPP, but I guess to formulate the regularized LDA in the framework of the theorem needs some work, such as making the estimated covariance matrix to be a vector, etc.

### pp. 111
4.2.8 talks about shrunken centroids classifier. If you check the actual code `shrunkenCentroidsFit` in the code for MLAPP, or some equivalent code in scikit-learn, you will see that it's computed using softthresholding, and no actual optimization criterion is given. pp. 651 (18.2) of ESLII describes the algorithm in more detail, and mentions that it can be formulated as a lasso-like problem (Ex. 18.2). But the procedure is not exactly formulated as a lasso. Actually, we can add some small (but boring) adjustments to the lasso formulation in Ex. 18.2 and its solution will be more like the the solution obtained via the actual algorithm, as shown in Eq. 5 of [Improved shrunken centroid classifiers for high-dimensional class-imbalanced data](http://dx.doi.org/10.1186/1471-2105-14-64).



*About Eq. (18.4) of ESLII:* I also checked the original shrunken centroids classifier paper [Diagnosis of multiple cancer types by shrunken centroids of gene expression](http://dx.doi.org/10.1073/pnas.082099299), apart from $s_0$ which I guess is just some parameter to make the algorithm work in that particular context, the reason we set $m_k$ to be $\sqrt{1/n_k - 1/n}$ (the paper is wrong; the package <http://statweb.stanford.edu/~tibs/PAM/Rdist/index.html> and the ESLII book all use minus instead of plus, although I think when $N$ is big, it won't matter; sklearn uses the wrong equation, see <https://github.com/scikit-learn/scikit-learn/issues/7679>) is shown as follows.

1. the paper assumes that we have several classes, and all these classes share the same diagonal covariance matrix. Thus (Eq. 2 of paper) we can estimate $s_i^2$ for the $i$th feature, by pooling all in-class variance. I believe $s_i^2$ is unbiased, and that $N-k$ is for correction. Thus, the estimated standard deviation is $s_i$.
2. The paper wants $d_{ik}$ to be some normalized measure of deviation of $i$th feature from the mean, for class $k$. To normalize it, we should make it zero mean and unit variance.
  * $\overline{x}_i$ is total mean feature over all classes, and is used to subtract mean from $\overline{x}_{ik}$ (well, I think $\overline{x}_i$ is the mean only when all classes are balanced; but let's assume this is true).
  * $m_k s_i$ is the estimated standard deviation of the numerator ($s_0$ is just numerical hack). To see this, we first notice that all sample points $x_{ij}$ (no matter what class $j$ is in) have esimated variance $s_i^2$ (since they come from some distribution with variance estimated as $s_i^2$, no matter of the class). Then we have
  $$
  \begin{align}
  \overline{x}_{ik} - \overline{x}_i &= \frac{\sum_{j \in C_k} x_{ij}}{n_k} - \frac{\sum_{q=1}^n x_{iq}}{n} \\
                                     &= \frac{n \sum_{j \in C_k} x_{ij}}{n n_k} - \frac{n_k \sum_{q=1}^n x_{iq}}{n n_k} \\
                                     &= \frac{1}{n n_k} [ (n - n_k) \sum_{j \in C_k} x_{ij} + n_k \sum_{j \notin C_k} x_{ij} ] \\
                                     &= \frac{n - n_k}{n n_k} \sum_{j \in C_k} x_{ij} + \frac{1}{n}\sum_{j \notin C_k} x_{ij}
  \end{align}
  $$
  * then basically, we have $n_k$ variables of weight $(n - n_k)(n n_k)$, and $n-n_k$ variables of weight $1/n$. Thus the variance of their weighted sum is sum of their weighted variance (since they are independent), we have
  $$
  \begin{align}
  (\frac{n - n_k}{n n_k})^2 n_k s_i^2 + (\frac{1}{n})^2 (n-n_k) s_i^2  & = (\frac{(n - n_k)^2}{n^2 n_k} + \frac{n-n_k}{n^2}) s_i^2 \\
  & = (\frac{(n - n_k)^2}{n^2 n_k} + \frac{n n_k-n_k^2}{n^2 n_k}) s_i^2 \\
  & = (\frac{n^2 + n_k^2 - 2n n_k}{n^2 n_k} + \frac{n n_k-n_k^2}{n^2 n_k}) s_i^2 \\
  & = (\frac{n^2 - n n_k}{n^2 n_k}) s_i^2 \\
  & = (\frac{n - n_k}{n n_k}) s_i^2 \\
  & = (\frac{1}{n_k} - \frac{1}{n}) s_i^2 
  \end{align}
  $$
  * take the square root and then we get the result.
3. The fact that we can solve a lasso by soft thresholding is consistent with my memory of convex optimization course where proximal methods involving L1 norm can be solved by soft thresholding. Basically I think if the lasso is decomposed to each individual variable, then it can be solved in one step via soft thresholding.

---

## 4.3 Inference in jointly Gaussian distributions

### pp. 115
I like this formulation of interpolation. From Eq. (4.76), we know the relationship between $\epsilon$ and the $x$ in the prior. Then (4.78) is basically saying that $\epsilon$ has form $N(0, (1/\lambda) I)$, and then replace $\epsilon$ by $Lx$. Although this prior itself is problematic, since $L$ doesn't have enough rank, in practice it works as long as you have enough (at least 2) data points. Notice that Eq. (4.82) on next page is wrong, and I think $L_1^{-1}$ should be its pseudoinverse. It's very interesting to compare this interpolation with Fig. 4.15 in pp. 127. This highlights the importance of our noise model.

---

## 4.6 Inferring the parameters of an MVN

### pp. 139
4.6.3.8 and 4.6.3.9 mention some equivalence between frequentist confidence interval and the corresponding Bayesian approach. BTW, I really don't understand why the author mentions paired test at all in 4.6.3.8, as what he simply works on the difference between paired samples, which is just one sample as well.